<a href="https://colab.research.google.com/github/Lokesh1854/Short-Term-Load-Forecasting/blob/main/pre_processing_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
import requests
from io import StringIO

# Define the Google Drive URL
url_weather = 'https://drive.google.com/uc?id=1I0jZHTrrLqMCO3XUJ0AoeLrvSMNhpKI3&export=download'
url_block0 = 'https://drive.google.com/uc?id=1ZceSLxN-d0BY32vDQ7jGw8svhLmQwCnW&export=download'

# Fetch the content of the file
response_weather = requests.get(url_weather)

# Create a StringIO object to read the content as a file-like object
csv_content_weather = StringIO(response_weather.text)

# Read the CSV content using pd.read_csv()
dfw = pd.read_csv(csv_content_weather)




In [ ]:
# Fetch the content of the file
response_block0 = requests.get(url_block0)

# Create a StringIO object to read the content as a file-like object
csv_content_block0 = StringIO(response_block0.text)

# Read the CSV content using pd.read_csv()
dfb0 = pd.read_csv(csv_content_block0)


In [ ]:
dfw["time"] = pd.to_datetime(dfw["time"])

In [ ]:
dfw.set_index('time', inplace=True)

# Sort the DataFrame by the index (which is the 'time' column)
dfws = dfw.sort_values(by='time')


In [ ]:
datetime_index = dfws.index

# Generate a new index with intermediate timestamps between each existing entry
new_index = []
for i in range(len(datetime_index) - 1):
  current_time = datetime_index[i]
  next_time = datetime_index[i + 1]
  time_delta = pd.Timedelta(minutes=30)
  # Create a datetime object halfway between current and next time
  intermediate_time = current_time + time_delta
  # Append current, intermediate, and next time to the new index
  new_index.extend([current_time, intermediate_time])
# Add the last time from the original index
new_index.append(datetime_index[-1])

In [ ]:
new_index_df = pd.DataFrame({'timestamp': new_index})

# Left join dfws with new_index_df
merged_df = pd.merge(new_index_df, dfws, left_on='timestamp', right_index=True, how='left')

# If you want to fill missing values with NaN
merged_df.fillna(value=np.nan, inplace=True)


In [ ]:
columns_to_drop = ['precipType', 'icon', 'summary']
cleaned_df = merged_df.drop(columns=columns_to_drop)

In [ ]:
interpolated_df = cleaned_df.interpolate(method='linear', axis=0, limit_direction='both')

In [ ]:
n_unique = dfb0['LCLid'].nunique()

In [ ]:
dfb0["tstp"] = pd.to_datetime(dfb0["tstp"])
dfb0["date"] = dfb0["tstp"].dt.date
dfb0["time"] = dfb0["tstp"].dt.time

In [ ]:
import pandas as pd

def get_start_indices(df):
  """
  This function takes a DataFrame containing an 'LCLid' column and returns a dictionary
  where keys are unique LCLids and values are their first indices in the DataFrame.
  """

  # Assuming the column name is 'LCLid'
  # Adjust the column name if it differs in your DataFrame
  unique_lclids = df['LCLid'].unique()  # Get unique LCLids

  # Initialize an empty dictionary to store results
  start_indices = {}

  # Iterate through unique LCLids and find their first indices
  for lclid in unique_lclids:
    start_index = df[df['LCLid'] == lclid].index[0]  # Find first index for each LCLid
    start_indices[lclid] = start_index

  return start_indices

# Get the dictionary with starting indices for each LCLid
start_indices = get_start_indices(dfb0.copy())  # Avoid modifying original DataFrame

# Print the results
# for lclid, start_index in start_indices.items():
#   print(f"LCLid: {lclid}, Starting Index: {start_index}")



In [ ]:
arr=[]
arr1 = []
for value in start_indices.values():
  arr.append(value)
for i in range(len(arr)-1):
  arr1.append(arr[i+1]-1)
# for i in range(len(arr)-1):
#   print(arr[i]," ",arr1[i])
# print(len(arr))

In [ ]:
startdate=[]
enddate=[]
indices = []
for i in range(len(arr)-1):
  #print(arr[i]," ",dfb0.date[arr[i]]," ",dfb0.date[arr[i+1]-1])
  indices.append(arr[i])
  startdate.append(dfb0.date[arr[i]])
  enddate.append(dfb0.date[arr[i+1]-1])
# print(arr[len(arr)-1]," ",dfb0.date[len(arr)-1]," ",dfb0.date[len(dfb0)-1])
# print(max(startdate))
# print(min(enddate))

In [ ]:
startdatec=startdate.copy()
enddatec=enddate.copy()
from datetime import timedelta
startdatec.sort(reverse=True)
enddatec.sort()
days = enddatec[0]-startdatec[0]
i = 0
delta = timedelta(days=365, hours=0, minutes=0, seconds=0)
while(days<delta):
  i = i+1
  days = abs(enddatec[i]-startdatec[i])
  #print(days)

# print(i)
# print(abs(startdatec[i]-enddatec[i]))
# print(startdatec[i])
# print(enddatec[i])

In [ ]:
dfb0c = dfb0.copy()

In [ ]:
print(i)
p = 0
z = 0
y = 0
j = 0
##for j in range(n_unique-1):
z = arr[j]
y = arr1[j]-arr[j]
#print(y)
#print(dfb0c.date[y])
p = arr[j]
while(p<=arr1[j] and dfb0c.date[z]<startdatec[i]):
  z = z+1
while(p<=arr1[j] and dfb0c.date[y]>enddatec[i]):
  y = y-1
# print(y)
# print(dfb0c.date[y])
#print(z," ",dfb0c.date[z]," ",y," ",dfb0c.date[y])


5


In [ ]:
print(i)
p = 0
z = 0
y = 0
arrz=[]
arry=[]
for j in range(n_unique-1):
  z = arr[j]
  y = arr1[j]
  p = arr[j]
  while(p<=arr1[j] and dfb0c.date[z]<startdatec[i]):
    z = z+1
  while(p<=arr1[j] and dfb0c.date[y]>enddatec[i]):
    y = y-1
  arrz.append(z)
  arry.append(y)
  #print(z," ",dfb0c.date[z]," ",y," ",dfb0c.date[y])
  #print(z," ",y)

5


In [ ]:
combined=[]
c=0
while(c<len(arrz)):
  combined.append(arrz[c])
  combined.append(arry[c])
  c=c+1

In [ ]:
one_year = 12*30*24*2
threshold = one_year
def get_intersection(data, indices):
  result = []
  for i in range(0, len(indices), 2):
    start_index = combined[i]
    end_index = combined[i + 1]+1
    slice_data = data[start_index:end_index]
    row_count = len(slice_data)
    if(row_count>threshold):
      result.append(slice_data)
  return pd.concat(result, ignore_index=True)
intersected_data = get_intersection(dfb0c, combined)


num_unique=intersected_data['LCLid'].nunique()
#num_unique


In [ ]:
power_data = intersected_data
#power_data

In [ ]:
weather_data = interpolated_df
#weather_data

In [ ]:

merged_data = pd.merge(weather_data, power_data, left_on='timestamp', right_on='tstp', how='left')

# Reset the index to default integer index
merged_data.reset_index(drop=True, inplace=True)

pivoted_data = merged_data.pivot(index=['timestamp', 'visibility', 'windBearing', 'temperature', 'dewPoint', 'pressure', 'apparentTemperature', 'windSpeed', 'humidity'], columns='LCLid', values='energy(kWh/hh)')

# Reset the index to default integer index
pivoted_data.reset_index(inplace=True)

if np.nan in pivoted_data.columns:
  pivoted_data.drop(columns=[np.nan], inplace=True)
#pivoted_data


In [ ]:
start_date = pd.to_datetime(startdatec[i])
end_date = pd.to_datetime(enddatec[i])+pd.Timedelta(days=1)

# Convert timestamp column to datetime if it's not already in datetime format
pivoted_data['timestamp'] = pd.to_datetime(pivoted_data['timestamp'])

# Filter the DataFrame based on the date range
filtered_data = pivoted_data[(pivoted_data['timestamp'] >= start_date) & (pivoted_data['timestamp'] < end_date)]

# Reset index
filtered_data.reset_index(drop=True, inplace=True)

#filtered_data

In [ ]:
median_values = filtered_data.median()
filtered_data.fillna(median_values, inplace=True)

<ipython-input-83-39e8391c3d31>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data.fillna(median_values, inplace=True)


In [ ]:
smart_meter_columns = filtered_data.columns[9:]
filtered_data[smart_meter_columns] = filtered_data[smart_meter_columns].astype(float)
filtered_data['PowerConsumption'] = filtered_data[smart_meter_columns].sum(axis=1)
filtered_data = filtered_data.drop(smart_meter_columns, axis=1)

<ipython-input-84-3cdc379bbeb0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data[smart_meter_columns] = filtered_data[smart_meter_columns].astype(float)
<ipython-input-84-3cdc379bbeb0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['PowerConsumption'] = filtered_data[smart_meter_columns].sum(axis=1)


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
numerical_columns = filtered_data.columns[1:9]
scaler.fit(filtered_data[numerical_columns])
filtered_data[numerical_columns] = scaler.transform(filtered_data[numerical_columns])

In [ ]:
filtered_data.to_csv('modified_dataset_block0.csv', index = False)

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

